In [ ]:
import cProfile
import functools
import geopy
import itertools
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyproj
import requests
import scipy as sp
import rtree
import seaborn as sb
from scipy import signal
# import shapely
import shapely.geometry
%pylab inline

import data_munging

In [ ]:
rides, readings = data_munging.read_raw_data()
readings = data_munging.clean_readings(readings)
readings = data_munging.add_proj_to_readings(readings, data_munging.NAD83)

In [ ]:
reload(data_munging)

In [ ]:
chi_readings = data_munging.filter_readings_to_chicago(readings)

In [ ]:
chi_readings = data_munging.filter_to_good_readings(chi_readings)

In [ ]:
print rides.shape
print readings.shape
n, p = readings.shape

In [ ]:
chi_readings.plot(x='start_lon', y='start_lat', kind='scatter')
config = plt.gcf()
config.set_size_inches(12, 12)
plt.show()

In [ ]:
chi_readings.plot(x='start_x', y='start_y', kind='scatter')
config = plt.gcf()
config.set_size_inches(12, 12)
plt.plot
plt.show()

In [ ]:
max_lat = np.max(chi_readings['start_lat'])
max_lon = np.max(chi_readings['start_lon'])
min_lat = np.min(chi_readings['start_lat'])
min_lon = np.min(chi_readings['start_lon'])

In [ ]:
max_x = np.max(chi_readings['start_x'])
max_y = np.max(chi_readings['start_y'])
min_x = np.min(chi_readings['start_x'])
min_y = np.min(chi_readings['start_y'])

In [ ]:
x_range = max_x - min_x
y_range = max_y - min_y

In [ ]:
data_munging.calc_dist(max_lon, max_lat, min_lon, min_lat)

In [ ]:
north_south_meters = data_munging.calc_dist(max_lon, max_lat, min_lon, max_lat)
east_west_meters = data_munging.calc_dist(max_lon, max_lat, max_lon, min_lat)
print north_south_meters
print east_west_meters

In [ ]:
patch_size = 5

In [ ]:
north_south_patches = int(np.ceil(north_south_meters / patch_size))
east_west_patches = int(np.ceil(east_west_meters / patch_size))

In [ ]:
x_step = x_range / east_west_patches
y_step = y_range / north_south_patches

In [ ]:
rastered_pavement_quality = np.zeros((north_south_patches, east_west_patches))

In [ ]:
readings_idx = data_munging.insert_readings_rtree(chi_readings)

In [ ]:
chi_readings['abs_mean_over_speed'] = (chi_readings['abs_mean_total'] /
                                       chi_readings['gps_speed'])

In [ ]:
np.mean(chi_readings['abs_mean_over_speed'][[]])

In [ ]:
total_itrs = 0
for i, j in itertools.product(range(north_south_patches), range(east_west_patches)):
    total_itrs += 1
    if total_itrs % 1000000 == 0:
        print total_itrs
    patch_bb = (min_x + x_step * j, min_y + y_step * i,
                min_x + x_step * (j + 1), min_y + y_step * (i + 1))
    segs_in_patch = list(readings_idx.intersection(patch_bb))
    if len(segs_in_patch) > 0:
        rastered_pavement_quality[i, j] = np.mean(chi_readings['abs_mean_over_speed'][segs_in_patch])

In [ ]:
print north_south_patches
print east_west_patches

In [ ]:
print north_south_patches * east_west_patches

In [ ]:
for i, j in itertools.product(range(north_south_patches), range(east_west_patches)):
    patch_bb = (min_x + x_step * j, min_y + y_step * i,
                min_x + x_step * (j + 1), min_y + y_step * (i + 1))
    segs_in_patch = list(readings_idx.intersection(patch_bb))
    pavement_quality = np.mean(chi_readings['abs_mean_over_speed'][segs_in_patch])
    if pavement_quality is np.nan:
        pavement_quality = 0
    rastered_pavement_quality[i, j] = pavement_quality

In [ ]:
rastered_pavement_quality.

In [ ]:
chi_readings.plot(x='start_x', y='start_y', kind='scatter')
plt.plot([patch_bb[0], patch_bb[2] + 5000],
         [patch_bb[1], patch_bb[3] + 2000], color='red')
config = plt.gcf()
config.set_size_inches(12, 12)
plt.show()

In [ ]:
def raster_closure():
    for i, j in itertools.product(range(100), range(100)):
        patch_bb = (min_x + x_step * j, min_y + y_step * i,
                    min_x + x_step * (j + 1), min_y + y_step * (i + 1))
        segs_in_patch = list(readings_idx.intersection(patch_bb))
        if len(segs_in_patch) > 0:
            rastered_pavement_quality[i, j] = np.mean(chi_readings['abs_mean_over_speed'][segs_in_patch])

In [ ]:
a_profile = cProfile.Profile()

In [ ]:
ran_profile = a_profile.run('raster_closure()')

In [ ]:
ran_profile.print_stats()

In [ ]:
a

In [ ]:
print chi_readings['start_lon'][0]
print chi_readings['start_lat'][0]